# Seal Detection Pipeline
---

This jupyter notebook will go through assembling the main components of a complete pipeline for counting seals in high-resolution satellite imagery (figure 1, steps 3 and 4) and show some experimental results with different pipeline designs. The ultimate goal of this pipeline is to perform a pan-Antarctic pack-ice seal census. ** Running this code will require input satellite imagery and at least one GPU with >8GB of memory **

<br>

<img src="jupyter_notebook_images/Base Pipeline.png">

<br>







## Table of contents
---
* [Getting started](#intro)
    * [Setup](#setup)
    * [Visualize training set](#vis_imgs)
* [Pipeline 1 - Seal haulout detector](#1)
    * [Training](#1T)
    * [Validation](#1V)
    * [Ablation experiment](#1A)
* [Pipeline 1.1 - Seal haulout detector + count](#1.1)
    * [Training](#1.1T)
    * [Validation](#1.1V)
    * [Ablation experiment / testing](#1.1A)
* [Pipeline 1.2 - Seal haulout detector + single seal detector](#1.2)
    * [Training](#1.2T)
    * [Validation](#1.2V)
    * [Testing](#1.2A)

## Getting started<a name="intro"></a>
---

If you followed the *training_set_generation* jupyter notebook (also present in this repo), you should have training sets generated and hyperparameter sets to try out, and be ready to search for a best performing seal detection pipeline.  Output files in this repository are organized as follows: *'./{dest_folder}/{pipeline}/{model_settings}/{model_settings}_{file}'*

### Setup environment<a name="setup"></a>

Before training and validating model/hyperparameter combinations inside the pipelines, we need to load the required python modules and a few global variables. Running this script will also display a list of training classes.

In [1]:
# import required packages
import os
import rasterio
import pandas as pd
import numpy as np
import operator
from PIL import Image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from functools import reduce
from utils.model_library import * 

%matplotlib inline
mpl.rcParams['figure.dpi']= 400

# destination folder for saved models and model stats
dest_folder = 'saved_models_stable'

# save class names
class_names = sorted([subdir for subdir in os.listdir('./training_sets/training_set_vanilla/training')])

### Visualizing training images (Optional)<a name="vis_imgs"></a>

To get a better sense for what the training set is like, the next cell will display a few random images from the training classes. Displayed images are extracted from a pool of ~70000 training images. 

In [ ]:
# store images
images = []

# loop over labels
for label in class_names:
    for path, _, files in os.walk('./training_sets/training_set_vanilla/training/{}'.format(label)):
        files = np.random.choice(files, 5)
        for filename in files:
            images.append(np.asarray(Image.open(os.path.join(path, filename))))

images = np.array(images)

# display images 
ncols=len(class_names)
nindex, height, width, intensity = images.shape
nrows = nindex // ncols
# check if rows and columns can fit the number of images
assert nindex == nrows * ncols
result = (images.reshape(nrows, ncols, height, width, intensity)
          .swapaxes(1,2)
          .reshape(height*nrows, width*ncols, intensity))

plt.imshow(result)
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)
plt.show()
    


## Pipeline 1 - haulout detector<a name="1"></a>
---

The simplest pipeline we can use is one that just uses an object classification step to find seal haulouts or penguins colonies. The obvious downside for this approach is that we often have more than one seal in a haulout, which is hardly usefull if we are looking for a count. However, we will use this as a 'pre-preocessing' step, where we narrow down the totality of patches to the subset where the haulout detection CNN flagged groups of seals. To validate the usefulness of this preprocessing step we can compare results obtained with the full pipeline (i.e. haul out detector + count) to one that simply tries to count on all tiles. 

### Training<a name="1T"></a>

The first step to find a best performing model is to train different model setups using our training set. To keep track of which combinations we have tried, how well they performed and the specifics of each model setup, we will store results in folders (under './{dest_folder}') named after each specific model combination.

In [ ]:
# switch pipeline
pipeline = 'Pipeline1'

# generate model combinations
combinations_1 = {'model_architecture': ['Resnet18', 'Resnet34', 'Resnet50', 'Squeezenet11',
                                         'Alexnet', 'NasnetA', 'Densenet121','Densenet169', 
                                         'VGG16'],
                  'training_dir': ['training_set_vanilla'] * 9,
                  'hyperparameter_set': ['B'] * 9,
                  'cv_weights': ['NO'] * 9}

# read as a DataFrame
combinations_1 = pd.DataFrame(combinations_1)
                    
# create folders for resulting files
for row in combinations_1.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]              
    if not os.path.exists("./{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("./{}/{}/{}".format(dest_folder, pipeline, mdl)) 


We can then provide model combinations created above as arguments to the training script, *train_sealnet.py*. A list of required arguments can be displayed by running the cell below.

In [ ]:
%run train_sealnet.py -h

In [ ]:
# iterate over combinations

for row in combinations_1.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st, cv_wgt = row[1]['training_dir'], row[1]['model_architecture'], \
                                  row[1]['hyperparameter_set'], row[1]['cv_weights']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st --cv_weights=$cv_wgt \
                             --output_name=$out --pipeline=$pipeline \
                             --dest_folder=$dest_folder
      

### Validation<a name="1V"></a> 

We can now load the models we just trained to get measurements of precision and recall for all positive classes. For every model combination we trained, *validate_sealnet.py* will run a full validation round and write given label/correct label pairs to a .csv file. The resulting .csv file is then imported by an R script, *plot_confusion_matrix.R*, which saves a confusion matrix figure and a .csv spreadsheet with precision and recall for all classes of interest. 

In [ ]:
# DataFrame to combine all metrics 
comb_prec_recall = pd.DataFrame()
pipeline = 'Pipeline1'

# iterate over trained models
for row in combinations_1.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st, cv_wgt = row[1]['training_dir'], row[1]['model_architecture'],\
                                  row[1]['hyperparameter_set'], row[1]['cv_weights']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model file is available
    if "{}.tar".format(out) not in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} has not been trained yet'.format(out))
        continue
        
    else:
        print()
        !echo validating $out
        print()
        
        #run validation
        !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                    --hyperparameter_set=$hyp_st --model_name=$out \
                                    --pipeline=$pipeline --dest_folder=$dest_folder
        
        # extract performance metrics and plot confusion matrix
        !Rscript plot_confusion_matrix.R --input_file=$out --pipeline=$pipeline \
                                         --dest_folder=$dest_folder
        
        # accumulate performance scores
        comb_prec_recall = comb_prec_recall.append(pd.read_csv('./{}/{}/{}/{}_prec_recall.csv'.format(dest_folder, pipeline, out, out)))
    
    
# Write combined metrics to csv and plot combined metrics
pooled_data_path = './{}/{}/pooled_prec_recall.csv'.format(dest_folder, pipeline)
comb_prec_recall.to_csv(pooled_data_path)
output_file_path = './{}/{}/comparison_plot.png'.format(dest_folder, pipeline) 
!Rscript plot_comparison.R --input_file=$pooled_data_path \
                           --output_file=$output_file_path \
                           --x='recall' --y='precision' --facet='label'
    

### Ablation experiment -- 11 classes training set vs. binary training set<a name="1A"></a>

Here we test how models with 11 classes compare with a model that simply classifies between 'seal' and 'not_seal'. We will train all architectures from Pipeline 1 on the binary training set and compare validation results to decide which approach is superior.  We begin by creating new model combinations which will be similar to the ones above but now using training_set_binary. 

In [ ]:
# switch pipeline 
pipeline = 'Pipeline1'

# generate model combinations -- now with training_set_binary
combinations_1_bin = {'model_architecture': ['Resnet18', 'Resnet34', 'Resnet50', 'Squeezenet11',
                                             'Alexnet', 'NasnetA', 'Densenet121','Densenet169', 
                                             'VGG16'],
                      'training_dir': ['training_set_binary'] * 9,
                      'hyperparameter_set': ['B'] * 9,
                      'cv_weights': ['NO'] * 9}

# read as a DataFrame
combinations_1_bin = pd.DataFrame(combinations_1_bin)
                    
# create folders for resulting files
for row in combinations_1_bin.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]             
    if not os.path.exists("./{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("./{}/{}/{}".format(dest_folder, pipeline, mdl)) 



### Training

In [ ]:
# iterate over combinations -- now with training_set_binary
for row in combinations_1_bin.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st, cv_wgt = row[1]['training_dir'], row[1]['model_architecture'], \
                                  row[1]['hyperparameter_set'], row[1]['cv_weights']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st --cv_weights=$cv_wgt \
                             --output_name=$out --pipeline=$pipeline \
                             --dest_folder=$dest_folder


### Validation

In [ ]:
# DataFrame to combine all metrics 
comb_prec_recall = pd.DataFrame()
pipeline = 'Pipeline1'

# iterate over trained models
for row in combinations_1_bin.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st, cv_wgt = row[1]['training_dir'], row[1]['model_architecture'],\
                                  row[1]['hyperparameter_set'], row[1]['cv_weights']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model file is available
    if "{}.tar".format(out) not in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} has not been trained yet'.format(out))
        continue
        
    else:
        print()
        !echo validating $out
        print()
        
        #run validation
        !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                    --hyperparameter_set=$hyp_st --model_name=$out \
                                    --pipeline=$pipeline --dest_folder=$dest_folder
        
        # extract performance metrics and plot confusion matrix
        !Rscript plot_confusion_matrix.R --input_file=$out --pipeline=$pipeline \
                                         --dest_folder=$dest_folder
        
        # accumulate performance scores
        comb_prec_recall = comb_prec_recall.append(pd.read_csv('./{}/{}/{}/{}_prec_recall.csv'.format(dest_folder, pipeline, out, out)))
    
    
# Write combined metrics to csv and plot combined metrics
pooled_data_path = './{}/{}/pooled_prec_recall.csv'.format(dest_folder, pipeline)
comb_prec_recall.to_csv(pooled_data_path)
output_file_path = './{}/{}/comparison_plot_bin.png'.format(dest_folder, pipeline) 
!Rscript plot_comparison.R --input_file=$pooled_data_path \
                           --output_file=$output_file_path \
                           --x='recall' --y='precision' --facet='label'
    

### Comparison -- binary vs. 11 classes 

We can visually compare validation results from both approaches by examining comparison plots.  


In [ ]:
# increase dpi
mpl.rcParams['figure.dpi']= 2500

# read both plots 
comp11 = mpimg.imread('./{}/{}/comparison_plot.png'.format(dest_folder, pipeline))
comp2 = mpimg.imread('./{}/{}/comparison_plot_bin.png'.format(dest_folder, pipeline))

# plot them side by side
plt.subplot(1, 2, 1)
plt.title('11 classes')
plt.imshow(comp11)

cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)

plt.subplot(1, 2, 2)
plt.imshow(comp2)

cur_axes = plt.gca()
plt.title('2 classes')
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)

plt.show()

## Pipeline 1.1 - haulout detector + count<a name="1.1"></a>
---

Here we will generate seal counting CNNs, train them and validate them. Seal counting CNNs will be trained to minimize the mean squared error (MSE) between predicted counts and ground-truth counts. Though they will be trained and validated separately from the haul out detector (Pipeline 1), these approaches will be tested on top of the haul out detector and as standalones.

### Training<a name="1.1T"></a>

Similar to the previous pipeline, we will store results in folders (under './saved_models') named after each specific model combination for bookkeeping.

In [5]:
# switch pipeline
pipeline = 'Pipeline1.1'

# generate model combinations
combinations_11 = {'model_architecture': ['WideResnetCount', 'Resnet34count', \
                                          'Resnet18count', \
                                          'NasnetAcount', 'CountCeption'] * 2,
                   'training_dir': ['training_set_vanilla'] * 5 + ['training_set_binary'] * 5,
                   'hyperparameter_set': ['D'] * 2 + ['A'] * 4 + ['B'] * 4}       

# read as a DataFrame
combinations_11 = pd.DataFrame(combinations_11)
                    

# create folders for resulting files
for row in combinations_11.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]               
    if not os.path.exists("./{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("./{}/{}/{}".format(dest_folder, pipeline, mdl)) 

To train a counting model, model combinations created above are used as argument to to a new training script, *train_sealnet_count.py*, which uses MSE loss. It accepts the same arguments as the previous.

In [6]:
# iterate over combinations
for row in combinations_11.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet_count.py --training_dir=$t_dir --model_architecture=$arch \
                                   --hyperparameter_set=$hyp_st --output_name=$out  \
                                   --pipeline=$pipeline --dest_folder=$dest_folder
    


training WideResnetCount_ts-vanilla

Epoch 1/5
----------
Traceback (most recent call last):
  File "train_sealnet_count.py", line 236, in <module>
    main()
  File "train_sealnet_count.py", line 232, in main
    num_epochs=hyperparameters[args.hyperparameter_set]['epochs'])
  File "train_sealnet_count.py", line 146, in train_model
    for data in dataloaders[phase]:
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 286, in __next__
    return self._process_next_batch(batch)
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 307, in _process_next_batch
    raise batch.exc_type(batch.exc_msg)
OSError: Traceback (most recent call last):
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.

UnboundLocalError: local variable 'child' referenced before assignment

### Validation<a name="1.1V"></a>

Validating counting models is a little bit simpler then with seal haul out models: for each model we just extract the mean squared error, running time at inference and number of model parameters. To test if classifying images before counting is helpful, performance stats during counting validation will be later compared to those where images where classified prior to counting.


In [ ]:
# DataFrame to combine all metrics 
comb_mse = pd.DataFrame()
pipeline = 'Pipeline1.1'

# iterate over trained models
for row in combinations_11.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'],\
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model file is available
    if "{}.tar".format(out) not in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} has not been trained yet'.format(out))
        continue
    
    else:
        print()
        !echo validating $out
        print()
        
        #run validation
        !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                    --hyperparameter_set=$hyp_st --model_name=$out \
                                    --pipeline=$pipeline --dest_folder=$dest_folder
        
        # extract performance metrics and plot confusion matrix
        !Rscript get_mse.R --input_file=$out --pipeline=$pipeline --dest_folder=$dest_folder
    
        # accumulate performance scores
        comb_mse = comb_mse.append(pd.read_csv('./{}/{}/{}/{}_mse.csv'.format(dest_folder, pipeline, out, out)))
    
# generate counting benchmarks and add them to the combined metrics DataFrame
!Rscript generate_benchmarks.R --input_file=$out --pipeline=$pipeline \
                               --dest_folder=$dest_folder
comb_mse = comb_mse.append(pd.read_csv('./{}/{}/benchmarks.csv'.format(dest_folder, pipeline)))

# write combined metrics to csv and plot combined metrics
pooled_data_path = './{}/{}/pooled_mse.csv'.format(dest_folder, pipeline)
comb_mse.to_csv(pooled_data_path)

# total predicted vs ground-truth plot
output_file_path = './{}/{}/comparison_final_count.png'.format(dest_folder, pipeline)
!Rscript plot_comparison.R --input_file=$pooled_data_path \
                           --output_file=$output_file_path \
                           --x='total_predicted' --y='total_ground_truth'
        
# inference time vs. MSE plot
output_file_path = './{}/{}/comparison_mse.png'.format(dest_folder, pipeline)
!Rscript plot_comparison.R --input_file=$pooled_data_path \
                           --output_file=$output_file_path \
                           --x='running_time' --y='MSE'

# precision vs. recall plot
output_file_path = './{}/{}/comparison_prec_recall.png'.format(dest_folder, pipeline)
!Rscript plot_comparison.R --input_file=$pooled_data_path \
                           --output_file=$output_file_path \
                           --x='recall' --y='precision'

## Pipeline 1.2 - haulout detector + single seal detector<a name="1.2"></a>
---

Pipeline 1.2 adds an individual seal detection CNN on top of the seal haul out detector (Pipeline 1. Individual seal detection CNNs will be trained to localize detection points and minimize the MSE between the number of detections and ground-truth count. In this approach, counts will be obtained by adding up the number of detections. Though they will be trained and validated separately from the haul out detector (Pipeline 1), these approaches will be tested on top of the haul out detector and as standalones.

### Training<a name="1.2T"></a>


In [5]:
# switch pipeline
pipeline = 'Pipeline1.2'

# generate model combinations
combinations_12 = {'model_architecture': ['UnetDet', 'UnetDet'],
                   'training_dir': ['training_set_binary', 'training_set_vanilla'],
                   'hyperparameter_set': ['B', 'B']}       

# read as a DataFrame
combinations_12 = pd.DataFrame(combinations_12)
                    

# create folders for resulting files
for row in combinations_12.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]                  
    if not os.path.exists("./{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("./{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [9]:
# iterate over combinations
for row in combinations_12.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet_det.py --training_dir=$t_dir --model_architecture=$arch \
                                 --hyperparameter_set=$hyp_st --output_name=$out  \
                                 --pipeline=$pipeline --dest_folder=$dest_folder
    


training UnetDet_ts-binary

Epoch 1/5
----------

 0 training iterations
   Hubber loss: 0.7242593765258789
   Euclidean loss: 1.6564775685502544
   BCE loss: 1.1911379098892212
   total loss: 1.9153972864151

 200 training iterations
   Hubber loss: 2.3120179176330566
   Euclidean loss: 1.5596258375522867
   BCE loss: 0.17397159337997437
   total loss: 2.485989511013031

 400 training iterations
   Hubber loss: 2.015331506729126
   Euclidean loss: 1.3903650832746606
   BCE loss: 0.0333438515663147
   total loss: 2.0486753582954407

 600 training iterations
   Hubber loss: 0.898402988910675
   Euclidean loss: 1.1422966264437096
   BCE loss: 0.010145379230380058
   total loss: 0.9085483681410551

 800 training iterations
   Hubber loss: 1.0583045482635498
   Euclidean loss: 0.18857554237620533
   BCE loss: 0.005427421536296606
   total loss: 1.0637319697998464

 1000 training iterations
   Hubber loss: 0.5775986909866333
   Euclidean loss: 0.9949795570987857
   BCE loss: 0.003970695659


 5200 training iterations
   Hubber loss: 0.00021214125445112586
   Euclidean loss: 0.0
   BCE loss: 0.0001485606044298038
   total loss: 0.00036070185888092965

 5400 training iterations
   Hubber loss: 3.708488748088712e-06
   Euclidean loss: 0.0
   BCE loss: 0.00014953743084333837
   total loss: 0.00015324591959142708

 5600 training iterations
   Hubber loss: 0.0005204820772632957
   Euclidean loss: 0.0
   BCE loss: 0.00014151839422993362
   total loss: 0.0006620004714932293

 5800 training iterations
   Hubber loss: 0.0002311283751623705
   Euclidean loss: 0.0
   BCE loss: 0.00014435594493988901
   total loss: 0.0003754843201022595

 6000 training iterations
   Hubber loss: 0.0005760377389378846
   Euclidean loss: 0.0
   BCE loss: 0.0001784644409781322
   total loss: 0.0007545021799160168

 6200 training iterations
   Hubber loss: 5.467914525070228e-05
   Euclidean loss: 0.0
   BCE loss: 0.00014568286132998765
   total loss: 0.00020036200658068992

 6400 training iterations
   Hu


 3000 training iterations
   Hubber loss: 0.0010466458043083549
   Euclidean loss: 0.0
   BCE loss: 1.1982819160039071e-05
   total loss: 0.001058628623468394

 3200 training iterations
   Hubber loss: 0.00029239783179946244
   Euclidean loss: 0.0
   BCE loss: 9.57989595917752e-06
   total loss: 0.00030197772775863996

 3400 training iterations
   Hubber loss: 0.0017132139764726162
   Euclidean loss: 0.0
   BCE loss: 9.78340449364623e-06
   total loss: 0.0017229973809662624

 3600 training iterations
   Hubber loss: 0.02850513905286789
   Euclidean loss: 0.0
   BCE loss: 1.1090661246271338e-05
   total loss: 0.02851622971411416

 3800 training iterations
   Hubber loss: 0.0023850579746067524
   Euclidean loss: 0.0
   BCE loss: 9.991586921387352e-06
   total loss: 0.0023950495615281397

 4000 training iterations
   Hubber loss: 0.0015332995681092143
   Euclidean loss: 0.0
   BCE loss: 9.824499102251139e-06
   total loss: 0.0015431240672114654

 4200 training iterations
   Hubber loss: 


 800 training iterations
   Hubber loss: 0.11289433389902115
   Euclidean loss: 0.0
   BCE loss: 8.223415534303058e-06
   total loss: 0.11290255731455545

 1000 training iterations
   Hubber loss: 0.013072723522782326
   Euclidean loss: 0.0
   BCE loss: 3.218685378669761e-06
   total loss: 0.013075942208160996

 1200 training iterations
   Hubber loss: 0.012860178016126156
   Euclidean loss: 0.0
   BCE loss: 1.4992148180681397e-06
   total loss: 0.012861677230944224

 1400 training iterations
   Hubber loss: 0.012913922779262066
   Euclidean loss: 0.0
   BCE loss: 1.0292751539964229e-06
   total loss: 0.012914952054416062

 1600 training iterations
   Hubber loss: 0.3078731894493103
   Euclidean loss: 0.0
   BCE loss: 1.0297922017343808e-05
   total loss: 0.30788348737132765

 1800 training iterations
   Hubber loss: 0.012925496324896812
   Euclidean loss: 0.0
   BCE loss: 4.587844614434289e-06
   total loss: 0.012930084169511247

 2000 training iterations
   Hubber loss: 0.0128500657


 3200 training iterations
   Hubber loss: 0.4388287663459778
   Euclidean loss: 0.6122187900918922
   BCE loss: 0.00014530037879012525
   total loss: 0.4389740667247679

 3400 training iterations
   Hubber loss: 0.03901855647563934
   Euclidean loss: 0.28381182664441623
   BCE loss: 6.925973866600543e-05
   total loss: 0.03908781621430535

 3600 training iterations
   Hubber loss: 0.32879167795181274
   Euclidean loss: 0.34764733386719165
   BCE loss: 0.0001843722420744598
   total loss: 0.3289760501938872

 3800 training iterations
   Hubber loss: 0.142726868391037
   Euclidean loss: 0.34785078340905623
   BCE loss: 0.00011606147745624185
   total loss: 0.14284292986849323

 4000 training iterations
   Hubber loss: 0.10890240222215652
   Euclidean loss: 0.17873675440257802
   BCE loss: 5.503498323378153e-05
   total loss: 0.1089574372053903

 4200 training iterations
   Hubber loss: 0.5211203098297119
   Euclidean loss: 0.33780703337279117
   BCE loss: 0.00012491292727645487
   total


 1000 training iterations
   Hubber loss: 0.35412511229515076
   Euclidean loss: 0.30648268323463773
   BCE loss: 9.007938206195831e-05
   total loss: 0.3542151916772127

 1200 training iterations
   Hubber loss: 0.21008449792861938
   Euclidean loss: 0.5085668387208913
   BCE loss: 8.732505375519395e-05
   total loss: 0.21017182298237458

 1400 training iterations
   Hubber loss: 0.23055928945541382
   Euclidean loss: 0.4525804117138987
   BCE loss: 0.00012092375982319936
   total loss: 0.23068021321523702

 1600 training iterations
   Hubber loss: 0.18587930500507355
   Euclidean loss: 0.2356374582785201
   BCE loss: 9.4414601335302e-05
   total loss: 0.18597371960640885

 1800 training iterations
   Hubber loss: 0.014428911730647087
   Euclidean loss: 0.14406135888745852
   BCE loss: 3.76662501366809e-05
   total loss: 0.014466577980783768

 2000 training iterations
   Hubber loss: 0.07637704908847809
   Euclidean loss: 0.5055226765396192
   BCE loss: 8.618024730822071e-05
   total


 6800 training iterations
   Hubber loss: 0.00030907863401807845
   Euclidean loss: 0.0
   BCE loss: 7.294500846910523e-06
   total loss: 0.00031637313486498897

 7000 training iterations
   Hubber loss: 0.05805201455950737
   Euclidean loss: 0.0
   BCE loss: 8.687548870511819e-06
   total loss: 0.05806070210837788

 7200 training iterations
   Hubber loss: 0.03420364111661911
   Euclidean loss: 0.0
   BCE loss: 1.4586008546757512e-05
   total loss: 0.03421822712516587

 7400 training iterations
   Hubber loss: 0.004011834505945444
   Euclidean loss: 0.0
   BCE loss: 0.0001287074846914038
   total loss: 0.004140541990636848

 7600 training iterations
   Hubber loss: 0.2608555853366852
   Euclidean loss: 0.15175308305060195
   BCE loss: 0.00035882348311133683
   total loss: 0.2612144088197965
validation Loss: 0.1399
training time: 3.0h 40m 47s

Training complete in 3.0h 40m 47s


### Validation<a name="1.2V"></a>

### Testing -- ablation study (real scenes)<a name="1.2T"></a>

Here we will test the output from top performing pipelines on real imagery. The following cell loops through a set of WV03 rasters, split them into patches and tries both classifying and counting and just counting. Finally, we can compare model predictions to those obtained with human observers. 

In [3]:
# create combinations for testing just counting
combs_count = pd.DataFrame({'arch': ['NasnetAcount', 'CountCeption',
                                     'Resnet18count', 'WideResnetCount'],
                            'hyp_st': ['B'] * 2 + ['A'] * 2})

# and just detecting
combs_det = pd.DataFrame({'arch': ['UnetDet'],
                          'hyp_st': ['B']})

# iterate through test images
test_imgs = [img for img in os.listdir('./test_scenes') if img[-4:] == '.tif']
for img in test_imgs:
    img_path = './test_scenes/{}'.format(img)
    
    # iterate through count CNN combinations
    for row in combs_count.iterrows():
        arch = row[1]['arch']
        hyp_st = row[1]['hyp_st']
        out_fldr = 'test_scenes/{}'.format(arch)
        # get results for counting
        !python predict_raster_det.py --input_image=$img_path \
                                      --class_architecture='NasnetA' --count_architecture=$arch \
                                      --hyperparameter_set_class='B' --hyperparameter_set_count=$hyp_st \
                                      --training_dir='training_set_binary' --dest_folder=$out_fldr \
                                      --skip_class='1'
    
    # iterate through detection CNN combinations
    for row in combs_det.iterrows():
        arch = row[1]['arch']
        hyp_st = row[1]['hyp_st']
        out_fldr = 'test_scenes/{}'.format(arch)
        # get results for detection
        !python predict_raster_det.py --input_image=$img_path \
                                      --class_architecture='NasnetA' --det_architecture=$arch \
                                      --hyperparameter_set_class='B' --hyperparameter_set_count=$hyp_st \
                                      --training_dir='training_set_binary' --dest_folder=$out_fldr \
                                      --skip_class='1'

    


13662 tiles created in 0 minutes and 23.54 seconds

Predicting with NasnetAcount:
Traceback (most recent call last):
  File "predict_raster_det.py", line 279, in <module>
    main()
  File "predict_raster_det.py", line 211, in main
    torch.load("./saved_models_stable/Pipeline1.1/{}/{}.tar".format(model_name, model_name)))
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/serialization.py", line 301, in load
    f = open(f, 'rb')
FileNotFoundError: [Errno 2] No such file or directory: './saved_models_stable/Pipeline1.1/NasnetAcount_ts-binary/NasnetAcount_ts-binary.tar'

13662 tiles created in 0 minutes and 22.30 seconds

Predicting with CountCeption:
Traceback (most recent call last):
  File "predict_raster_det.py", line 279, in <module>
    main()
  File "predict_raster_det.py", line 211, in main
    torch.load("./saved_models_stable/Pipeline1.1/{}/{}.tar".format(model_name, model_name)))
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/serialization.py"

#### Classifying + counting

In [ ]:
# create combinations for testing count + classify
combinations_class_count = pd.DataFrame()
combinations_class_det = pd.DataFrame()

# define architectures we wish to test
class_archs = ['NasnetA', 'Densenet169']
count_archs = ['WideResnetCount', 'CountCeption']
det_archs = ['UnetDet']
t_sets = ['training_set_vanilla', 'training_set_binary']
hyp_sets = {'NasnetA': 'B', 'Densenet169': 'B', 'CountCeption': 'B',
            'WideResnetCount': 'A', 'UnetDet': 'B'}

# add combinations 
for class_arch in class_archs:
    for t_set in t_sets:
        for count_arch in count_archs:
            combinations_class_count = combinations_class_count.append(
                pd.Series({'class_arch': class_arch, 'count_arch': count_arch, 
                           'hyp_class': hyp_sets[class_arch], 
                           'hyp_count': hyp_sets[count_arch],
                           't_set': t_set}), ignore_index=True)
        for det_arch in det_archs:
            combinations_det_count = combinations_det_count.append(
                pd.Series({'class_arch': class_arch, 'det_arch': count_arch, 
                           'hyp_class': hyp_sets[class_arch], 
                           'hyp_count': hyp_sets[count_arch],
                           't_set': t_set}), ignore_index=True)
        

# iterate through test images
test_imgs = [img for img in os.listdir('./test_scenes') if img[-4:] == '.tif']
for img in test_imgs:
    img_path = 'test_scenes/{}'.format(img)
    # iterate through combinations -- counting
    for row in combinations_class_count.iterrows():
        cnt_arch = row[1]['count_arch']
        cls_arch = row[1]['class_arch']
        hyp_cnt = row[1]['hyp_count']
        hyp_cls = row[1]['hyp_class']
        t_set = row[1]['t_set']
        # find positive classes
        if t_set.split('_')[-1] == 'binary':
            pos_classes = 'seal'
        else:
            pos_classes='crabeater_weddell'
        out_fldr = 'test_scenes/{}-{}-{}'.format(cls_arch, cnt_arch, t_set.split('_')[-1])
        # check if combination was already tried
        if os.path.exists('./{}/predicted_shapefiles/{}'.format(out_fldr, img[:-4])):
            print('Already classified {} with {}'.format(img, os.path.basename(out_fldr)))
            continue
        !python predict_raster_det.py --input_image=$img_path \
                                      --class_architecture=$cls_arch --count_architecture=$cnt_arch \
                                      --hyperparameter_set_class=$hyp_cls --hyperparameter_set_count=$hyp_cnt \
                                      --training_dir=$tset --dest_folder=$out_fldr \
                                      --pos_classes=$pos_classes
    # iterate through combinations -- detecting
    for row in combinations_class_det.iterrows():
        det_arch = row[1]['det_arch']
        cls_arch = row[1]['class_arch']
        hyp_cnt = row[1]['hyp_count']
        hyp_cls = row[1]['hyp_class']
        t_set = row[1]['t_set']
        # find positive classes
        if t_set.split('_')[-1] == 'binary':
            pos_classes = 'seal'
        else:
            pos_classes='crabeater_weddell'
        out_fldr = 'test_scenes/{}-{}'.format(cls_arch, cnt_arch, t_set.split('_')[-1])
        # check if combination was already tried
        if os.path.exists('./{}/predicted_shapefiles/{}'.format(out_fldr, img[:-4])):
            print('Already classified {} with {}'.format(img, os.path.basename(out_fldr)))
            continue
        !python predict_raster_det.py --input_image=$img_path \
                                      --class_architecture=$cls_arch --det_architecture=$det_arch \
                                      --hyperparameter_set_class=$hyp_cls --hyperparameter_set_count=$hyp_cnt \
                                      --training_dir=$tset --dest_folder=$out_fldr \
                                      --pos_classes=$pos_classes